In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))


In [ ]:
import pandas as pd
import numpy as np
import random
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from sklearn.linear_model import LinearRegression

In [ ]:
#Loading the train_df
#The training data path
train_path = 'https://raw.githubusercontent.com/Premalatha-success/Datasets/main/TaxiFare.csv'

#Defining constants
sample_fraction = 0.01
random.seed(92)

#Defining the function to randomly sample 1 percent of the data
def skip_row(idx):
    if idx == 0:
        return False
    return random.random() > sample_fraction

#Loading the df
df = pd.read_csv(train_path, parse_dates = ['date_time_of_pickup'], skiprows = skip_row)

In [ ]:
df.head()

,unique_id,amount,date_time_of_pickup,longitude_of_pickup,latitude_of_pickup,longitude_of_dropoff,latitude_of_dropoff,no_of_passenger
0,42:49.0,9.8,2009-03-02 20:42:49+00:00,-73.972673,40.759186,-73.969897,40.791367,1
1,19:05.0,11.5,2015-04-12 10:19:05+00:00,-73.979279,40.723438,-74.004608,40.746948,6
2,33:14.0,9.5,2012-11-06 11:33:14+00:00,-73.972195,40.765883,-73.967942,40.761951,1
3,47:00.0,6.5,2012-10-06 15:47:00+00:00,-73.982130,40.774807,-73.973583,40.789645,5
4,33:18.0,16.5,2015-05-31 07:33:18+00:00,-73.996986,40.755428,-73.953194,40.802101,3


In [ ]:
#Loading the test_df
#The test data path
test_path = 'https://raw.githubusercontent.com/sravani999999/taxi-fare-csv/main/test.csv'

#Loading the df
test_df = pd.read_csv(test_path, parse_dates = ['date_time_of_pickup'],)

In [ ]:
#Viewing info about the training data
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 506 entries, 0 to 505
Data columns (total 8 columns):
 #   Column                Non-Null Count  Dtype              
---  ------                --------------  -----              
 0   unique_id             506 non-null    object             
 1   amount                506 non-null    float64            
 2   date_time_of_pickup   506 non-null    datetime64[ns, UTC]
 3   longitude_of_pickup   506 non-null    float64            
 4   latitude_of_pickup    506 non-null    float64            
 5   longitude_of_dropoff  506 non-null    float64            
 6   latitude_of_dropoff   506 non-null    float64            
 7   no_of_passenger       506 non-null    int64              
dtypes: datetime64[ns, UTC](1), float64(5), int64(1), object(1)
memory usage: 31.8+ KB


In [ ]:
#Statististical summary of the training datset
df.describe()

,amount,longitude_of_pickup,latitude_of_pickup,longitude_of_dropoff,latitude_of_dropoff,no_of_passenger
count,506.00000,506.000000,506.000000,506.000000,506.000000,506.000000
mean,10.92834,-72.805135,40.108897,-72.656774,40.027111,1.620553
std,10.50899,9.236903,5.088683,9.787082,5.391785,1.217973
min,2.50000,-74.027978,0.000000,-74.177225,0.000000,0.000000
25%,6.00000,-73.991434,40.739385,-73.990350,40.735169,1.000000
50%,8.10000,-73.981695,40.755563,-73.978803,40.755624,1.000000
75%,11.50000,-73.967273,40.768157,-73.963257,40.768767,2.000000
max,143.00000,0.000000,40.827673,0.000000,40.851849,6.000000


In [ ]:
#Checking the min and highest date in the training df
df['date_time_of_pickup'].min(), df['date_time_of_pickup'].max()

(Timestamp('2009-01-01 17:07:00+0000', tz='UTC'),
 Timestamp('2015-06-24 07:43:40+0000', tz='UTC'))

In [ ]:
#info on the test set
test_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9914 entries, 0 to 9913
Data columns (total 7 columns):
 #   Column                Non-Null Count  Dtype              
---  ------                --------------  -----              
 0   unique_id             9914 non-null   object             
 1   date_time_of_pickup   9914 non-null   datetime64[ns, UTC]
 2   longitude_of_pickup   9914 non-null   float64            
 3   latitude_of_pickup    9914 non-null   float64            
 4   longitude_of_dropoff  9914 non-null   float64            
 5   latitude_of_dropoff   9914 non-null   float64            
 6   no_of_passenger       9914 non-null   int64              
dtypes: datetime64[ns, UTC](1), float64(4), int64(1), object(1)
memory usage: 542.3+ KB


In [ ]:
#Statistical summary of the test data
test_df.describe()

,longitude_of_pickup,latitude_of_pickup,longitude_of_dropoff,latitude_of_dropoff,no_of_passenger
count,9914.000000,9914.000000,9914.000000,9914.000000,9914.000000
mean,-73.974722,40.751041,-73.973657,40.751743,1.671273
std,0.042774,0.033541,0.039072,0.035435,1.278747
min,-74.252193,40.573143,-74.263242,40.568973,1.000000
25%,-73.992501,40.736125,-73.991247,40.735254,1.000000
50%,-73.982326,40.753051,-73.980015,40.754065,1.000000
75%,-73.968013,40.767113,-73.964059,40.768757,2.000000
max,-72.986532,41.709555,-72.990963,41.696683,6.000000


In [ ]:
#Checking the min and highest date in the test df
test_df['date_time_of_pickup'].min(), test_df['date_time_of_pickup'].max()

(Timestamp('2009-01-01 11:04:24+0000', tz='UTC'),
 Timestamp('2015-06-30 20:03:50+0000', tz='UTC'))

In [ ]:
#Viewing the colums in the df
df.columns

Index(['unique_id', 'amount', 'date_time_of_pickup', 'longitude_of_pickup',
       'latitude_of_pickup', 'longitude_of_dropoff', 'latitude_of_dropoff',
       'no_of_passenger'],
      dtype='object')

In [ ]:
#Dropping missng Values
df = df.dropna()

In [ ]:
df.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 506 entries, 0 to 505
Data columns (total 8 columns):
 #   Column                Non-Null Count  Dtype              
---  ------                --------------  -----              
 0   unique_id             506 non-null    object             
 1   amount                506 non-null    float64            
 2   date_time_of_pickup   506 non-null    datetime64[ns, UTC]
 3   longitude_of_pickup   506 non-null    float64            
 4   latitude_of_pickup    506 non-null    float64            
 5   longitude_of_dropoff  506 non-null    float64            
 6   latitude_of_dropoff   506 non-null    float64            
 7   no_of_passenger       506 non-null    int64              
dtypes: datetime64[ns, UTC](1), float64(5), int64(1), object(1)
memory usage: 31.8+ KB


In [ ]:
#Splitting the training data into features and labels
X = df.drop(['unique_id', 'amount','date_time_of_pickup'], axis = 1)
y = df['amount']

In [ ]:
#Subsetting the test data
test = test_df.drop(['unique_id','date_time_of_pickup'], axis = 1)

In [ ]:
#Splitting the data into training and validation set
X_train, X_val, y_train, y_val = train_test_split(X,y,test_size = 0.2, random_state = 92)

In [ ]:
#Validating the split
print(X_train.shape, X_val.shape)
print(y_train.shape, y_val.shape)

(404, 5) (102, 5)
(404,) (102,)


In [ ]:
#Mean Regressor
class MeanRegressor:
    def fit(self, inputs, targets):
        self.mean = targets.mean()

    def predict(self, inputs):
        return np.full(inputs.shape[0], self.mean)

In [ ]:
#Instantiating the model
mean_model = MeanRegressor()

#Fitting the model to the input data
mean_model.fit(X_train, y_train)

#Predicting on the train data
train_pred = mean_model.predict(X_train)

#Predicting on the val data
val_pred = mean_model.predict(X_val)

In [ ]:
#Defining the evaluation function

def rmse(targets, pred):
    return mean_squared_error(targets, pred, squared = False)

In [ ]:
#Train rmse
train_rmse = rmse(y_train, train_pred)


#Validation rmse
val_rmse = rmse(y_val, val_pred)

print(f"The Root mean squared error of the baseline model on the train data is: {train_rmse}")
print(f"The Root mean squared erorr of the baseline model on the val data is: {val_rmse}")

The Root mean squared error of the baseline model on the train data is: 11.08865823551579
The Root mean squared erorr of the baseline model on the val data is: 7.731058654074997


Training a Linear Regression Model

In [ ]:
#Instantiating the model
l_reg = LinearRegression()

In [ ]:
#Fitting the model to the train data
l_reg.fit(X_train,y_train)

#Making predictions on the train data
train_pred = l_reg.predict(X_train)

#Making predictions on the val data
val_pred = l_reg.predict(X_val)

In [ ]:
#RMSE on the train prediction
train_rmse = rmse(y_train, train_pred)

#RMSE on the val prediction
val_rmse = rmse(y_val, val_pred)

print(f"The RMSE of the model on the train data is: {train_rmse}")
print(f"The RMSE of the model on the val data is: {val_rmse}")

The RMSE of the model on the train data is: 10.214961705630069
The RMSE of the model on the val data is: 44.171571939946055


In [ ]:
#Making predictions on the test data
test_pred = l_reg.predict(test)

In [ ]:
#Preparing the submission file
sub_dir = 'https://raw.githubusercontent.com/sravani999999/taxi-fare-csv/main/sample_submission.csv'
submission = pd.read_csv(sub_dir)

#Viewing the file
submission

,unique_id,amount
0,08:24.0,11.35
1,08:24.0,11.35
2,53:44.0,11.35
3,12:12.0,11.35
4,12:12.0,11.35
...,...,...
9909,37:51.0,11.35
9910,05:51.0,11.35
9911,44:15.0,11.35
9912,05:19.0,11.35


In [ ]:
#Subtituting the values
submission['fare_amount'] = test_pred

In [ ]:
#Saving the file to csv
submission.to_csv('1st_submission.csv', index = None)

In [ ]:
#Defining a function for predicting and saving the submission file
def make_predictions_and_submit(model, test_input, file_name):
    test_pred = model.predict(test_input)
    sub_df = pd.read_csv(sub_dir)
    sub_df['amount'] = test_pred
    sub_df.to_csv(file_name, index = None)
    return sub_df

In [ ]:
import pandas as pd

# Read the CSV file into a pandas dataframe
submission = pd.read_csv('1st_submission.csv')

# View the first few rows of the dataframe
print(submission.head())


  unique_id  amount  fare_amount
0   08:24.0   11.35    12.323274
1   08:24.0   11.35     5.903753
2   53:44.0   11.35    10.289392
3   12:12.0   11.35    12.303971
4   12:12.0   11.35    15.830014


In [ ]:
print(submission)

     unique_id  amount  fare_amount
0      08:24.0   11.35    12.323274
1      08:24.0   11.35     5.903753
2      53:44.0   11.35    10.289392
3      12:12.0   11.35    12.303971
4      12:12.0   11.35    15.830014
...        ...     ...          ...
9909   37:51.0   11.35    15.369402
9910   05:51.0   11.35    17.453056
9911   44:15.0   11.35     6.032419
9912   05:19.0   11.35     7.244756
9913   06:23.0   11.35     8.300591

[9914 rows x 3 columns]
